# Web Scraping Players' Stats in the Premier League

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import time, os
import requests

In [2]:
url = 'https://fbref.com/robots.txt'
response = requests.get(url)

In [3]:
url2 = 'https://www.spotrac.com/epl/manchester-united-f.c/payroll/2018/robots.txt'
response2 = requests.get(url2)

In [4]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

# 2018-2019 Season Stats

## Manchester United 

### Standard Stats

In [5]:
def standardtable(teamURL):
    '''
    input: teamURL (str)
    output: dataframe of the standard stats table with selected stats
    '''
    #URL html for standard table read in beautifulsoup
    page = requests.get(teamURL).text
    soup = BeautifulSoup(page, 'lxml')

    #finding standard stats table
    standard_table = soup.find('table', id='stats_standard_ks_1889')

    #player names in list
    players = [
        header for header in standard_table.find_all('th', class_='left')
    ]
    player_names = [name.text for th in players for name in th.find_all('a')]

    #player stats in list
    player_row = [row for row in standard_table.find_all('tr')]

    player_stats = {}
    for player in player_row[2:]:
        items = player.find_all('td')
        player_stats[player] = [i.text for i in items]

    player_stats_list = [stats for stats in player_stats.values()]

    #matching player name with stats
    all_players_stats = {}
    all_players_stats.update(zip(player_names, player_stats_list))

    #creating dataframe for team's players and stats
    team_1819 = pd.DataFrame(all_players_stats).T
    team_1819.columns = [
        'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'PK',
        'PKatt', 'CrdY', 'CrdR', 'P90Gls', 'P90Ast', 'P90G+A', 'P90G-PK',
        'P90G+A-PK', 'ExpectedxG', 'ExpectednpxG', 'ExpectedxA', 'P90xG',
        'P90xA', 'P90xG+xA', 'P90npxG', 'P90npxG+xA', 'Matches'
    ]

    #dropping unneded columns
    team_1819 = team_1819.drop([
        'Pos', 'Age', 'Nation', 'PK', 'PKatt', 'CrdY', 'CrdR', 'P90Gls',
        'P90G+A', 'P90G-PK', 'P90G+A-PK', 'ExpectedxG', 'ExpectednpxG',
        'ExpectedxA', 'P90xG', 'P90xA', 'P90xG+xA', 'P90npxG', 'P90npxG+xA',
        'Matches', 'P90Ast'
    ],
                               axis=1)

    #data cleaning
    team_1819 = team_1819.replace(r'^\s*$', np.nan, regex=True)
    team_1819['Min'] = team_1819['Min'].str.replace(',', '')
    team_1819.iloc[:, :] = team_1819.iloc[:, :].astype('float64')

    return team_1819

In [6]:
standardtable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')

,MP,Starts,Min,Gls,Ast
David de Gea,38.0,38.0,3420.0,0.0,0.0
Paul Pogba,35.0,34.0,3006.0,13.0,9.0
Victor Lindelöf,30.0,29.0,2601.0,1.0,1.0
Luke Shaw,29.0,29.0,2591.0,1.0,4.0
Ashley Young,30.0,28.0,2569.0,2.0,2.0
Nemanja Matić,28.0,28.0,2436.0,1.0,0.0
Marcus Rashford,33.0,26.0,2334.0,10.0,6.0
Chris Smalling,24.0,24.0,2127.0,1.0,0.0
Romelu Lukaku,32.0,22.0,2136.0,12.0,0.0
Jesse Lingard,27.0,19.0,1663.0,4.0,2.0


### Shooting Stats

In [7]:
from bs4 import BeautifulSoup, Comment

In [8]:
def shootingtable(teamURL):
    '''
    input: teamURL (str)
    output: dataframe of the shooting stats table with selected stats
    '''
    #URL html for shooting table read in beautifulsoup
    page = requests.get(teamURL).text
    soup = BeautifulSoup(page, 'lxml')
    placeholder = soup.select_one('#all_kitchen_sink_shooting .placeholder')
    comment = next(elem for elem in placeholder.next_siblings
                   if isinstance(elem, Comment))
    shootsoup = BeautifulSoup(comment, 'lxml')

    #finding shooting stats table
    shooting_table = shootsoup.find('table', id="stats_shooting_ks_1889")

    #player names in list
    shooters = [
        header for header in shooting_table.find_all('th', class_='left')
    ]
    shooter_names = [name.text for th in shooters for name in th.find_all('a')]

    #player stats in list
    shooting_row = [row for row in shooting_table.find_all('tr')]

    shooting_stats = {}
    for shooter in shooting_row[2:]:
        items = shooter.find_all('td')
        shooting_stats[shooter] = [i.text for i in items]
    shooting_stats_list = [stats for stats in shooting_stats.values()]

    #matching player name with stats
    all_shooting = {}
    all_shooting.update(zip(shooter_names, shooting_stats_list))

    #creating dataframe for team's players and stats
    shooting_df = pd.DataFrame(all_shooting).T
    shooting_df.columns = [
        'Nation', 'Pos', 'Age', '90s', 'Gls', 'PK', 'Pkatt', 'Shot',
        'ShotonTarget', 'FK', 'ShotonTarget%', 'Sh/90', 'SoT/90', 'G/Sh',
        'G/SoT', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG', 'Matches'
    ]

    #dropping unneded columns
    shooting_df = shooting_df.drop([
        'Nation', 'Pos', 'Age', '90s', 'Gls', 'PK', 'Pkatt', 'Sh/90', 'SoT/90',
        'G/Sh', 'G/SoT', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG', 'Matches',
        'Shot', 'ShotonTarget%', 'FK'
    ],
                                   axis=1)

    #data cleaning
    shooting_df = shooting_df.replace(r'^\s*$', np.nan, regex=True)
    shooting_df.iloc[:, :] = shooting_df.iloc[:, :].astype('float64')

    return shooting_df

In [9]:
shootingtable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')

,ShotonTarget
David de Gea,0.0
Paul Pogba,38.0
Victor Lindelöf,1.0
Luke Shaw,6.0
Ashley Young,3.0
Nemanja Matić,3.0
Marcus Rashford,40.0
Chris Smalling,4.0
Romelu Lukaku,33.0
Jesse Lingard,12.0


### Passing Stats

In [10]:
def passingtable(teamURL):
    '''
    input: teamURL(str)
    output: dataframe of the passing stats table with selected stats
    '''
    #URL html for shooting table read in beautifulsoup
    page = requests.get(teamURL).text
    soup = BeautifulSoup(page, 'lxml')
    placeholder = soup.select_one('#all_kitchen_sink_passing .placeholder')
    comment = next(elem for elem in placeholder.next_siblings
                   if isinstance(elem, Comment))
    passsoup = BeautifulSoup(comment, 'lxml')

    #finding shooting stats table
    passing_table = passsoup.find('table', id='stats_passing_ks_1889')

    #player names in list
    passers = [
        header for header in passing_table.find_all('th', class_='left')
    ]
    passer_names = [name.text for th in passers for name in th.find_all('a')]

    #player stats in list
    passing_row = [row for row in passing_table.find_all('tr')]

    passing_stats = {}
    for passer in passing_row[2:]:
        items = passer.find_all('td')
        passing_stats[passer] = [i.text for i in items]
    passing_stats_list = [stats for stats in passing_stats.values()]

    #matching player name with stats
    all_passing = {}
    all_passing.update(zip(passer_names, passing_stats_list))

    #creating dataframe for team's players and stats
    passing_df = pd.DataFrame(all_passing).T
    passing_df.columns = [
        'Nation', 'Pos', 'Age', '90s', 'TotPassCmp', 'TotPassAtt',
        'TotPassCmpPerc', 'TotDist', 'TotPrgDist', 'ShortCmp', 'ShortAtt',
        'ShortCmp%', 'MediumCmp', 'MediumAtt', 'MediumCmp%', 'LongCmp',
        'LongAtt', 'LongCmp%', 'Ast', 'xA', 'A-xA', 'KP', '1/3', 'PPA',
        'CrsPA', 'Prog', 'Matches'
    ]

    #dropping unneded columns
    passing_df = passing_df.drop([
        'Nation', 'Pos', 'Age', '90s', 'TotDist', 'ShortCmp', 'ShortAtt',
        'ShortCmp%', 'MediumCmp', 'MediumAtt', 'MediumCmp%', 'LongCmp',
        'LongAtt', 'LongCmp%', 'Ast', 'xA', 'A-xA', 'KP', '1/3', 'PPA',
        'CrsPA', 'TotPrgDist', 'Matches', 'TotPassAtt', 'TotPassCmpPerc'
    ],
                                 axis=1)

    #data cleaning
    passing_df = passing_df.replace(r'^\s*$', np.nan, regex=True)
    passing_df.iloc[:, :] = passing_df.iloc[:, :].astype('float64')

    return passing_df

In [11]:
passingtable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')

,TotPassCmp,Prog
David de Gea,712.0,1.0
Paul Pogba,1728.0,274.0
Victor Lindelöf,1323.0,71.0
Luke Shaw,1657.0,238.0
Ashley Young,1380.0,259.0
Nemanja Matić,1579.0,152.0
Marcus Rashford,538.0,75.0
Chris Smalling,800.0,38.0
Romelu Lukaku,379.0,49.0
Jesse Lingard,668.0,49.0


### Possession Stats

In [12]:
def possessiontable(teamURL):
    '''
    input: teamURL(str)
    output: dataframe of the possession stats table with selected stats
    '''
    #URL html for shooting table read in beautifulsoup
    page = requests.get(teamURL).text
    soup = BeautifulSoup(page, 'lxml')
    placeholder = soup.select_one('#all_kitchen_sink_possession .placeholder')
    comment = next(elem for elem in placeholder.next_siblings
                   if isinstance(elem, Comment))
    possessionsoup = BeautifulSoup(comment, 'lxml')

    #finding shooting stats table
    possession_table = possessionsoup.find('table',
                                           id='stats_possession_ks_1889')

    #player names in list
    possessers = [
        header for header in possession_table.find_all('th', class_='left')
    ]
    possesser_names = [
        name.text for th in possessers for name in th.find_all('a')
    ]

    #player stats in list
    possession_row = [row for row in possession_table.find_all('tr')]

    possession_stats = {}
    for possesser in possession_row[2:]:
        items = possesser.find_all('td')
        possession_stats[possesser] = [i.text for i in items]
    possession_stats_list = [stats for stats in possession_stats.values()]

    #matching player name with stats
    all_possession = {}
    all_possession.update(zip(possesser_names, possession_stats_list))

    #creating dataframe for team's players and stats
    possession_df = pd.DataFrame(all_possession).T
    possession_df.columns = [
        'Nation', 'Pos', 'Age', '90s', 'Touches', 'Def Pen', 'Def 3rd',
        'Mid 3rd', 'Touches_Att3rd', 'Att Pen', 'Live', 'DribSucc', 'Att',
        'DribSuccPerc', 'DribPl', 'Megs', 'Carries', 'TotDist', 'PrgDist',
        'RecTarg', 'Rec', 'RecTargPerc', 'Miscon', 'Dispos', 'Matches'
    ]

    #dropping unneded columns
    possession_df = possession_df.drop([
        'Nation', 'Pos', 'Age', '90s', 'Touches', 'Def Pen', 'Def 3rd',
        'Mid 3rd', 'Att Pen', 'Live', 'DribSuccPerc', 'Att', 'Megs', 'Carries',
        'TotDist', 'PrgDist', 'RecTarg', 'RecTargPerc', 'Miscon', 'Dispos', 'Matches'
    ],
                                       axis=1)

    #data cleaning
    possession_df = possession_df.replace(r'^\s*$', np.nan, regex=True)
    possession_df.iloc[:, :] = possession_df.iloc[:, :].astype('float64')

    return possession_df

In [13]:
possessiontable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')

,Touches_Att3rd,DribSucc,DribPl,Rec
David de Gea,1.0,0.0,0.0,422.0
Paul Pogba,870.0,60.0,62.0,2075.0
Victor Lindelöf,38.0,7.0,7.0,1142.0
Luke Shaw,656.0,28.0,30.0,1371.0
Ashley Young,649.0,22.0,23.0,1185.0
Nemanja Matić,314.0,18.0,20.0,1410.0
Marcus Rashford,660.0,48.0,52.0,851.0
Chris Smalling,42.0,1.0,1.0,668.0
Romelu Lukaku,467.0,26.0,27.0,653.0
Jesse Lingard,449.0,19.0,23.0,765.0


### Player Salaries

In [14]:
from unidecode import unidecode 

def salarytable(teamURL2):
    '''
    input: teamURL2 (str)
    output: dataframe of the salary table of selected team
    '''
    #URL html for salary table read in beautifulsoup
    page = requests.get(teamURL2).text
    soup = BeautifulSoup(page, 'lxml')

    #finding salary stats table
    salary_table = soup.find('table', attrs={'class': 'datatable rtable'})

    #player salaries in list
    player_row = [row for row in salary_table.find_all('tr')]

    player_salary = {}
    for player in player_row[1:]:
        items = player.find_all('td')
        player_name = items[0].text
        player_salary[player_name] = [i.text for i in items[1:]]

    #creating dataframe for team's players and salaries
    team_1819 = pd.DataFrame(player_salary).T
    team_1819.columns = [
        'Pos', 'Age', 'Blank', 'Annual_Salary(GBP)', 'Weekly_Salary'
    ]

    #dropping unneded columns
    team_1819 = team_1819.drop(['Pos', 'Age', 'Blank', 'Weekly_Salary'],
                               axis=1)

    #data cleaning
    team_1819 = team_1819.reset_index()
    team_1819['Annual_Salary(GBP)'] = team_1819['Annual_Salary(GBP)'].replace(
        '[^\w\s]', '', regex=True)
    team_1819['index'] = team_1819['index'].str.split(
        '\n', n=1).str[1].str.rstrip().apply(unidecode).str.lower()

    team_1819.set_index('index', inplace=True)

    return team_1819

In [15]:
salarytable('https://www.spotrac.com/epl/manchester-united-f.c/payroll/2018/')

,Annual_Salary(GBP)
index,
alexis sanchez,18200000
paul pogba,15080000
david de gea,10400000
romelu lukaku,9360000
juan mata,8320000
luke shaw,7800000
frederico de paula santos,6240000
victor lindelof,6240000
nemanja matic,6240000


### Final Team Stats

In [16]:
manutd_standard = standardtable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')
manutd_shooting = shootingtable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')
manutd_passing = passingtable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')
manutd_possession = possessiontable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')
manutd_salary = salarytable(
    'https://www.spotrac.com/epl/manchester-united-f.c/payroll/2018/')

In [17]:
def teamfinaltable(team_standard, team_shooting, team_passing, team_possession, team_salary):
    '''
    input: team_standard, team_shooting, team_passing, team_possession (dataframes)
    output: merge input into one final table (dataframe)
    '''
    teamfinal_1819 = team_standard.join(team_shooting)
    teamfinal_1819 = teamfinal_1819.join(team_passing)
    teamfinal_1819 = teamfinal_1819.join(team_possession)
    
    #standardizing all letters - getting rid of accents, etc.
    teamfinal_1819 = teamfinal_1819.reset_index()
    teamfinal_1819['index'] = teamfinal_1819['index'].apply(unidecode).str.lower()
    teamfinal_1819.set_index('index', inplace=True)
    
    teamfinal_1819 = teamfinal_1819.join(team_salary)

    return teamfinal_1819

In [18]:
#merging manutd tables into one table
manutd_final = teamfinaltable(manutd_standard, manutd_shooting, manutd_passing,
                              manutd_possession, manutd_salary)

In [19]:
manutd_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
david de gea,38.0,38.0,3420.0,0.0,0.0,0.0,712.0,1.0,1.0,0.0,0.0,422.0,10400000
paul pogba,35.0,34.0,3006.0,13.0,9.0,38.0,1728.0,274.0,870.0,60.0,62.0,2075.0,15080000
victor lindelof,30.0,29.0,2601.0,1.0,1.0,1.0,1323.0,71.0,38.0,7.0,7.0,1142.0,6240000
luke shaw,29.0,29.0,2591.0,1.0,4.0,6.0,1657.0,238.0,656.0,28.0,30.0,1371.0,7800000
ashley young,30.0,28.0,2569.0,2.0,2.0,3.0,1380.0,259.0,649.0,22.0,23.0,1185.0,6240000
nemanja matic,28.0,28.0,2436.0,1.0,0.0,3.0,1579.0,152.0,314.0,18.0,20.0,1410.0,6240000
marcus rashford,33.0,26.0,2334.0,10.0,6.0,40.0,538.0,75.0,660.0,48.0,52.0,851.0,2340000
chris smalling,24.0,24.0,2127.0,1.0,0.0,4.0,800.0,38.0,42.0,1.0,1.0,668.0,4160000
romelu lukaku,32.0,22.0,2136.0,12.0,0.0,33.0,379.0,49.0,467.0,26.0,27.0,653.0,9360000


## Manchester City

### Final Team Stats

In [20]:
mancity_standard = standardtable(
    'https://fbref.com/en/squads/b8fd03ef/2018-2019/Manchester-City-Stats')
mancity_shooting = shootingtable(
    'https://fbref.com/en/squads/b8fd03ef/2018-2019/Manchester-City-Stats')
mancity_passing = passingtable(
    'https://fbref.com/en/squads/b8fd03ef/2018-2019/Manchester-City-Stats')
mancity_possession = possessiontable(
    'https://fbref.com/en/squads/b8fd03ef/2018-2019/Manchester-City-Stats')
mancity_salary = salarytable(
    'https://www.spotrac.com/epl/manchester-city-f.c/payroll/2018/')

In [21]:
#merging mancity tables into one table
mancity_final = teamfinaltable(mancity_standard, mancity_shooting,
                               mancity_passing, mancity_possession, mancity_salary)

In [22]:
mancity_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
ederson,38.0,38.0,3420.0,0.0,1.0,0.0,918.0,3.0,0.0,1.0,1.0,628.0,NaN
aymeric laporte,35.0,34.0,3057.0,3.0,3.0,7.0,2793.0,251.0,208.0,2.0,3.0,2602.0,3380000
bernardo silva,36.0,31.0,2854.0,7.0,7.0,22.0,1513.0,168.0,1141.0,50.0,55.0,1628.0,6240000
raheem sterling,34.0,31.0,2771.0,17.0,9.0,37.0,1099.0,100.0,1237.0,86.0,91.0,1453.0,9360000
sergio aguero,33.0,31.0,2459.0,21.0,8.0,41.0,665.0,68.0,779.0,29.0,32.0,892.0,13000000
kyle walker,33.0,30.0,2779.0,1.0,1.0,3.0,2337.0,189.0,753.0,23.0,25.0,2128.0,5200000
david silva,33.0,28.0,2401.0,6.0,8.0,18.0,1783.0,204.0,1174.0,27.0,31.0,1911.0,11440000
fernandinho,29.0,27.0,2377.0,1.0,3.0,7.0,1804.0,218.0,447.0,8.0,12.0,1760.0,NaN
ilkay gundogan,31.0,23.0,2137.0,6.0,3.0,11.0,1837.0,201.0,824.0,17.0,18.0,1806.0,6240000


## Liverpool

### Final Team Stats

In [23]:
liverpool_standard = standardtable(
    'https://fbref.com/en/squads/822bd0ba/2018-2019/Liverpool-Stats')
liverpool_shooting = shootingtable(
    'https://fbref.com/en/squads/822bd0ba/2018-2019/Liverpool-Stats')
liverpool_passing = passingtable(
    'https://fbref.com/en/squads/822bd0ba/2018-2019/Liverpool-Stats')
liverpool_possession = possessiontable(
    'https://fbref.com/en/squads/822bd0ba/2018-2019/Liverpool-Stats')
liverpool_salary = salarytable(
    'https://www.spotrac.com/epl/liverpool-f.c/payroll/2018/')

In [24]:
#merging liverpool tables into one table
liverpool_final = teamfinaltable(liverpool_standard, liverpool_shooting,
                                 liverpool_passing, liverpool_possession, liverpool_salary)

In [25]:
liverpool_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
alisson,38.0,38.0,3420.0,0.0,0.0,0.0,1039.0,0.0,0.0,1.0,1.0,671.0,NaN
virgil van dijk,38.0,38.0,3384.0,4.0,2.0,8.0,2709.0,157.0,126.0,7.0,8.0,2410.0,9360000
mohamed salah,38.0,37.0,3250.0,22.0,8.0,60.0,864.0,158.0,1173.0,79.0,86.0,1301.0,10400000
andrew robertson,36.0,36.0,3216.0,0.0,11.0,5.0,2283.0,321.0,1041.0,30.0,32.0,2155.0,2600000
sadio mane,36.0,35.0,3077.0,22.0,1.0,42.0,932.0,111.0,962.0,61.0,67.0,1323.0,5200000
georginio wijnaldum,35.0,32.0,2725.0,3.0,0.0,8.0,1620.0,131.0,439.0,33.0,36.0,1546.0,3900000
roberto firmino,34.0,31.0,2607.0,12.0,6.0,31.0,1056.0,158.0,830.0,54.0,60.0,1327.0,9360000
trent alexander-arnold,29.0,27.0,2462.0,1.0,12.0,9.0,1644.0,322.0,862.0,20.0,23.0,1543.0,NaN
fabinho,28.0,21.0,2017.0,1.0,2.0,4.0,1403.0,135.0,324.0,14.0,16.0,1349.0,NaN


## Chelsea

### Final Team Stats

In [26]:
chelsea_standard = standardtable(
    'https://fbref.com/en/squads/cff3d9bb/2018-2019/Chelsea-Stats')
chelsea_shooting = shootingtable(
    'https://fbref.com/en/squads/cff3d9bb/2018-2019/Chelsea-Stats')
chelsea_passing = passingtable(
    'https://fbref.com/en/squads/cff3d9bb/2018-2019/Chelsea-Stats')
chelsea_possession = possessiontable(
    'https://fbref.com/en/squads/cff3d9bb/2018-2019/Chelsea-Stats')
chelsea_salary = salarytable(
    'https://www.spotrac.com/epl/chelsea-f.c/payroll/2018/')

In [27]:
#merging chelsea tables into one table
chelsea_final = teamfinaltable(chelsea_standard, chelsea_shooting,
                               chelsea_passing, chelsea_possession, chelsea_salary)

In [28]:
chelsea_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
cesar azpilicueta,38.0,38.0,3402.0,1.0,5.0,7.0,2471.0,231.0,904.0,16.0,17.0,2144.0,4680000
jorginho,37.0,37.0,3156.0,2.0,0.0,6.0,2703.0,317.0,682.0,15.0,16.0,2612.0,NaN
kepa arrizabalaga,36.0,36.0,3240.0,0.0,0.0,0.0,974.0,1.0,0.0,1.0,1.0,623.0,7800000
david luiz,36.0,36.0,3238.0,3.0,2.0,8.0,2293.0,232.0,137.0,8.0,8.0,2143.0,6240000
n'golo kante,36.0,36.0,3092.0,4.0,4.0,6.0,1805.0,150.0,920.0,32.0,36.0,1792.0,7500000
antonio rudiger,33.0,33.0,2871.0,1.0,0.0,7.0,2310.0,149.0,76.0,4.0,5.0,2079.0,6000000
eden hazard,37.0,32.0,2923.0,16.0,15.0,39.0,1595.0,269.0,1673.0,134.0,143.0,2082.0,11700000
marcos alonso,31.0,31.0,2759.0,2.0,4.0,7.0,1838.0,175.0,903.0,22.0,25.0,1706.0,5200000
willian,32.0,26.0,2097.0,3.0,6.0,22.0,1100.0,186.0,1008.0,37.0,41.0,1295.0,NaN


## Tottenham

### Final Team Stats

In [29]:
tottenham_standard = standardtable(
    'https://fbref.com/en/squads/361ca564/2018-2019/Tottenham-Hotspur-Stats')
tottenham_shooting = shootingtable(
    'https://fbref.com/en/squads/361ca564/2018-2019/Tottenham-Hotspur-Stats')
tottenham_passing = passingtable(
    'https://fbref.com/en/squads/361ca564/2018-2019/Tottenham-Hotspur-Stats')
tottenham_possession = possessiontable(
    'https://fbref.com/en/squads/361ca564/2018-2019/Tottenham-Hotspur-Stats')
tottenham_salary = salarytable(
    'https://www.spotrac.com/epl/tottenham-hotspur-f.c/payroll/2018/')

In [30]:
#merging tottenham tables into one table
tottenham_final = teamfinaltable(tottenham_standard, tottenham_shooting,
                                 tottenham_passing, tottenham_possession, tottenham_salary)

In [31]:
tottenham_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
hugo lloris,33.0,33.0,2970.0,0.0,0.0,0.0,890.0,0.0,0.0,0.0,0.0,634.0,5200000
toby alderweireld,34.0,33.0,2922.0,0.0,0.0,6.0,2067.0,133.0,115.0,2.0,2.0,1899.0,NaN
christian eriksen,35.0,30.0,2774.0,8.0,12.0,31.0,1540.0,228.0,931.0,21.0,23.0,1692.0,3900000
harry kane,28.0,27.0,2424.0,17.0,4.0,42.0,423.0,81.0,511.0,43.0,45.0,743.0,10400000
moussa sissoko,29.0,27.0,2330.0,0.0,3.0,3.0,1205.0,102.0,412.0,45.0,49.0,1185.0,4160000
kieran trippier,27.0,26.0,2276.0,1.0,3.0,3.0,1515.0,262.0,818.0,9.0,10.0,1346.0,3380000
lucas moura,32.0,25.0,2121.0,10.0,0.0,20.0,561.0,61.0,552.0,44.0,46.0,791.0,4160000
son heung-min,31.0,23.0,2039.0,12.0,6.0,28.0,646.0,71.0,662.0,55.0,62.0,858.0,NaN
davinson sanchez,23.0,22.0,1942.0,1.0,1.0,3.0,1289.0,52.0,39.0,0.0,0.0,1152.0,3380000


## Arsenal

### Final Team Stats

In [32]:
arsenal_standard = standardtable(
    'https://fbref.com/en/squads/18bb7c10/2018-2019/Arsenal-Stats')
arsenal_shooting = shootingtable(
    'https://fbref.com/en/squads/18bb7c10/2018-2019/Arsenal-Stats')
arsenal_passing = passingtable(
    'https://fbref.com/en/squads/18bb7c10/2018-2019/Arsenal-Stats')
arsenal_possession = possessiontable(
    'https://fbref.com/en/squads/18bb7c10/2018-2019/Arsenal-Stats')
arsenal_salary = salarytable(
    'https://www.spotrac.com/epl/arsenal-f.c/payroll/2018/')

In [33]:
#merging arsenal tables into one table
arsenal_final = teamfinaltable(arsenal_standard, arsenal_shooting,
                               arsenal_passing, arsenal_possession, arsenal_salary)

In [34]:
arsenal_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
bernd leno,32.0,31.0,2836.0,0.0,0.0,0.0,779.0,2.0,0.0,0.0,0.0,521.0,5200000
shkodran mustafi,31.0,31.0,2611.0,2.0,0.0,9.0,1438.0,105.0,119.0,12.0,12.0,1369.0,4680000
pierre-emerick aubameyang,36.0,30.0,2726.0,22.0,5.0,34.0,526.0,68.0,600.0,29.0,32.0,831.0,10400000
granit xhaka,29.0,29.0,2498.0,4.0,2.0,11.0,1955.0,272.0,510.0,16.0,16.0,1851.0,5200000
alexandre lacazette,35.0,27.0,2502.0,13.0,8.0,28.0,579.0,79.0,664.0,52.0,53.0,893.0,9467273
sokratis papastathopoulos,25.0,25.0,2198.0,1.0,2.0,1.0,1215.0,40.0,37.0,8.0,8.0,1129.0,4784000
lucas torreira,34.0,24.0,2382.0,2.0,2.0,6.0,1420.0,137.0,348.0,22.0,23.0,1279.0,3900000
matteo guendouzi,33.0,23.0,2149.0,0.0,0.0,3.0,1412.0,146.0,401.0,31.0,33.0,1395.0,2080000
alex iwobi,35.0,22.0,1970.0,3.0,6.0,14.0,774.0,130.0,692.0,53.0,58.0,1073.0,2600000


## Wolves

### Final Team Stats

In [35]:
wolves_standard = standardtable(
    'https://fbref.com/en/squads/8cec06e1/2018-2019/Wolverhampton-Wanderers-Stats'
)
wolves_shooting = shootingtable(
    'https://fbref.com/en/squads/8cec06e1/2018-2019/Wolverhampton-Wanderers-Stats'
)
wolves_passing = passingtable(
    'https://fbref.com/en/squads/8cec06e1/2018-2019/Wolverhampton-Wanderers-Stats'
)
wolves_possession = possessiontable(
    'https://fbref.com/en/squads/8cec06e1/2018-2019/Wolverhampton-Wanderers-Stats'
)
wolves_salary = salarytable(
    'https://www.spotrac.com/epl/wolverhampton-wanderers-f.c/payroll/2018/'
)

In [36]:
#merging wolves tables into one table
wolves_final = teamfinaltable(wolves_standard, wolves_shooting, wolves_passing,
                              wolves_possession, wolves_salary)

In [37]:
wolves_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
conor coady,38.0,38.0,3420.0,0.0,0.0,0.0,1412.0,61.0,8.0,1.0,2.0,1267.0,780000
rui patricio,37.0,37.0,3329.0,0.0,0.0,0.0,493.0,0.0,0.0,0.0,0.0,280.0,5200000
willy boly,36.0,36.0,3167.0,4.0,0.0,6.0,1392.0,125.0,91.0,17.0,17.0,1282.0,2080000
raul jimenez,38.0,36.0,3111.0,13.0,7.0,32.0,789.0,81.0,688.0,36.0,37.0,1133.0,2132000
matt doherty,38.0,35.0,3144.0,4.0,5.0,17.0,1333.0,294.0,867.0,27.0,30.0,1294.0,520000
joao moutinho,38.0,35.0,3032.0,1.0,8.0,3.0,1660.0,263.0,673.0,15.0,15.0,1467.0,NaN
ryan bennett,34.0,34.0,3060.0,1.0,0.0,3.0,1095.0,95.0,132.0,1.0,1.0,980.0,1040000
ruben neves,35.0,34.0,3007.0,4.0,3.0,15.0,1569.0,176.0,339.0,12.0,14.0,1503.0,2600000
jonny castro,33.0,32.0,2771.0,1.0,1.0,7.0,1124.0,107.0,653.0,26.0,27.0,955.0,1907000


## Everton

### Final Team Stats

In [38]:
everton_standard = standardtable(
    'https://fbref.com/en/squads/d3fd31cc/2018-2019/Everton-Stats')
everton_shooting = shootingtable(
    'https://fbref.com/en/squads/d3fd31cc/2018-2019/Everton-Stats')
everton_passing = passingtable(
    'https://fbref.com/en/squads/d3fd31cc/2018-2019/Everton-Stats')
everton_possession = possessiontable(
    'https://fbref.com/en/squads/d3fd31cc/2018-2019/Everton-Stats')
everton_salary = salarytable(
    'https://www.spotrac.com/epl/everton-f.c/payroll/2018/')

In [39]:
#merging everton tables into one table
everton_final = teamfinaltable(everton_standard, everton_shooting,
                               everton_passing, everton_possession, everton_salary)

In [40]:
everton_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
jordan pickford,38.0,38.0,3420.0,0.0,0.0,0.0,748.0,1.0,0.0,0.0,0.0,563.0,5200000
gylfi sigurdsson,38.0,36.0,3117.0,13.0,6.0,29.0,830.0,190.0,797.0,38.0,42.0,996.0,5200000
michael keane,33.0,33.0,2969.0,1.0,2.0,9.0,1285.0,108.0,81.0,5.0,7.0,1106.0,3120000
lucas digne,35.0,33.0,2965.0,4.0,4.0,10.0,1460.0,293.0,895.0,14.0,16.0,1205.0,4680000
idrissa gana gueye,33.0,32.0,2822.0,0.0,2.0,3.0,1478.0,171.0,361.0,22.0,23.0,1302.0,NaN
richarlison,35.0,32.0,2665.0,13.0,1.0,27.0,546.0,76.0,727.0,41.0,50.0,944.0,NaN
kurt zouma,32.0,29.0,2612.0,2.0,2.0,4.0,1218.0,87.0,85.0,4.0,5.0,1042.0,4000000
seamus coleman,29.0,29.0,2558.0,2.0,2.0,5.0,1218.0,188.0,583.0,25.0,26.0,1027.0,3640000
bernard,34.0,25.0,2116.0,1.0,3.0,5.0,633.0,91.0,557.0,45.0,48.0,821.0,NaN


## Leicester City

### Final Team Stats

In [41]:
leicester_standard = standardtable(
    'https://fbref.com/en/squads/a2d435b3/2018-2019/Leicester-City-Stats')
leicester_shooting = shootingtable(
    'https://fbref.com/en/squads/a2d435b3/2018-2019/Leicester-City-Stats')
leicester_passing = passingtable(
    'https://fbref.com/en/squads/a2d435b3/2018-2019/Leicester-City-Stats')
leicester_possession = possessiontable(
    'https://fbref.com/en/squads/a2d435b3/2018-2019/Leicester-City-Stats')
leicester_salary = salarytable(
    'https://www.spotrac.com/epl/leicester-city/payroll/2018/')

In [42]:
#merging leicester tables into one table
leicester_final = teamfinaltable(leicester_standard, leicester_shooting,
                                 leicester_passing, leicester_possession, leicester_salary)

In [43]:
leicester_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
kasper schmeichel,38.0,38.0,3420.0,0.0,0.0,0.0,825.0,3.0,2.0,2.0,2.0,538.0,6760000
wilfred ndidi,38.0,37.0,3296.0,2.0,0.0,9.0,1634.0,153.0,399.0,40.0,45.0,1533.0,NaN
ben chilwell,36.0,36.0,3240.0,0.0,4.0,10.0,1575.0,211.0,969.0,37.0,44.0,1624.0,3380000
ricardo pereira,35.0,35.0,3123.0,2.0,6.0,9.0,1495.0,224.0,767.0,62.0,72.0,1409.0,3640000
james maddison,36.0,35.0,2831.0,7.0,7.0,27.0,1243.0,237.0,946.0,43.0,46.0,1394.0,2860000
harry maguire,31.0,31.0,2596.0,3.0,0.0,11.0,1496.0,166.0,192.0,17.0,17.0,1299.0,4680000
jamie vardy,34.0,30.0,2726.0,18.0,4.0,32.0,286.0,38.0,403.0,19.0,21.0,490.0,7280000
nampalys mendy,31.0,23.0,2092.0,0.0,0.0,0.0,969.0,80.0,200.0,19.0,21.0,870.0,2600000
demarai gray,34.0,23.0,1985.0,4.0,1.0,19.0,523.0,46.0,488.0,46.0,46.0,733.0,2860000


## West Ham

### Final Team Stats

In [44]:
westham_standard = standardtable(
    'https://fbref.com/en/squads/7c21e445/2018-2019/West-Ham-United-Stats')
westham_shooting = shootingtable(
    'https://fbref.com/en/squads/7c21e445/2018-2019/West-Ham-United-Stats')
westham_passing = passingtable(
    'https://fbref.com/en/squads/7c21e445/2018-2019/West-Ham-United-Stats')
westham_possession = possessiontable(
    'https://fbref.com/en/squads/7c21e445/2018-2019/West-Ham-United-Stats')
westham_salary = salarytable(
    'https://www.spotrac.com/epl/west-ham-united-f.c/payroll/2018/')

In [45]:
#merging westham tables into one table
westham_final = teamfinaltable(westham_standard, westham_shooting,
                               westham_passing, westham_possession, westham_salary)

In [46]:
westham_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
lukasz fabianski,38.0,38.0,3420.0,0.0,0.0,0.0,651.0,0.0,1.0,1.0,1.0,417.0,3380000
felipe anderson,36.0,36.0,3043.0,9.0,4.0,24.0,1328.0,223.0,1039.0,84.0,89.0,1650.0,4420000
declan rice,34.0,34.0,3003.0,2.0,0.0,5.0,1364.0,85.0,219.0,25.0,25.0,1078.0,156000
issa diop,33.0,33.0,2970.0,1.0,0.0,4.0,973.0,64.0,29.0,5.0,5.0,801.0,3484000
mark noble,31.0,29.0,2291.0,5.0,5.0,5.0,1293.0,163.0,375.0,15.0,16.0,1287.0,2600000
robert snodgrass,33.0,25.0,2220.0,2.0,5.0,8.0,875.0,138.0,710.0,34.0,37.0,973.0,2080000
marko arnautovic,28.0,24.0,1990.0,10.0,4.0,30.0,496.0,46.0,541.0,28.0,29.0,691.0,5200000
pablo zabaleta,26.0,23.0,2092.0,0.0,1.0,1.0,863.0,145.0,409.0,10.0,10.0,714.0,3900000
fabian balbuena,23.0,23.0,2013.0,1.0,0.0,2.0,655.0,42.0,41.0,2.0,2.0,584.0,2080000


## Watford

### Final Team Stats

In [47]:
watford_standard = standardtable(
    'https://fbref.com/en/squads/2abfe087/2018-2019/Watford-Stats')
watford_shooting = shootingtable(
    'https://fbref.com/en/squads/2abfe087/2018-2019/Watford-Stats')
watford_passing = passingtable(
    'https://fbref.com/en/squads/2abfe087/2018-2019/Watford-Stats')
watford_possession = possessiontable(
    'https://fbref.com/en/squads/2abfe087/2018-2019/Watford-Stats')
watford_salary = salarytable(
    'https://www.spotrac.com/epl/watford/payroll/2018/')

In [48]:
#merging watford tables into one table
watford_final = teamfinaltable(watford_standard, watford_shooting,
                               watford_passing, watford_possession, watford_salary)

In [49]:
watford_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
ben foster,38.0,38.0,3420.0,0.0,0.0,0.0,600.0,2.0,1.0,0.0,0.0,429.0,1560000
craig cathcart,36.0,35.0,3096.0,3.0,0.0,6.0,1046.0,64.0,58.0,3.0,3.0,874.0,1200000
abdoulaye doucoure,35.0,34.0,3062.0,5.0,6.0,16.0,1640.0,165.0,565.0,41.0,44.0,1578.0,3640000
etienne capoue,33.0,33.0,2944.0,1.0,3.0,3.0,1482.0,164.0,244.0,32.0,32.0,1285.0,2080000
roberto pereyra,33.0,33.0,2815.0,6.0,1.0,21.0,967.0,125.0,760.0,46.0,54.0,1109.0,2340000
will hughes,32.0,31.0,2418.0,2.0,4.0,8.0,773.0,108.0,573.0,21.0,23.0,921.0,1560000
troy deeney,32.0,28.0,2545.0,9.0,5.0,28.0,522.0,109.0,560.0,6.0,10.0,856.0,3380000
jose holebas,28.0,27.0,2393.0,3.0,6.0,6.0,819.0,175.0,562.0,23.0,25.0,618.0,1560000
gerard deulofeu,30.0,26.0,2066.0,10.0,5.0,23.0,580.0,85.0,668.0,44.0,46.0,802.0,3120000


## Crystal Palace

### Final Team Stats

In [50]:
crystalpalace_standard = standardtable(
    'https://fbref.com/en/squads/47c64c55/2018-2019/Crystal-Palace-Stats')
crystalpalace_shooting = shootingtable(
    'https://fbref.com/en/squads/47c64c55/2018-2019/Crystal-Palace-Stats')
crystalpalace_passing = passingtable(
    'https://fbref.com/en/squads/47c64c55/2018-2019/Crystal-Palace-Stats')
crystalpalace_possession = possessiontable(
    'https://fbref.com/en/squads/47c64c55/2018-2019/Crystal-Palace-Stats')
crystalpalace_salary = salarytable(
    'https://www.spotrac.com/epl/crystal-palace/payroll/2018/')

In [51]:
#merging crystalpalace tables into one table
crystalpalace_final = teamfinaltable(crystalpalace_standard,
                                     crystalpalace_shooting,
                                     crystalpalace_passing,
                                     crystalpalace_possession, crystalpalace_salary)

In [52]:
crystalpalace_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
luka milivojevic,38.0,38.0,3420.0,12.0,2.0,13.0,1629.0,248.0,545.0,18.0,20.0,1486.0,2860000
patrick van aanholt,36.0,36.0,3195.0,3.0,2.0,12.0,1587.0,292.0,682.0,27.0,28.0,1351.0,2860000
james mcarthur,38.0,36.0,3058.0,3.0,6.0,8.0,1151.0,101.0,472.0,29.0,30.0,1141.0,2860000
aaron wan-bissaka,35.0,35.0,3134.0,0.0,3.0,0.0,1156.0,164.0,558.0,75.0,78.0,933.0,780000
wilfried zaha,34.0,34.0,3036.0,10.0,5.0,30.0,839.0,147.0,945.0,143.0,155.0,1339.0,6760000
andros townsend,38.0,34.0,3009.0,6.0,4.0,17.0,895.0,134.0,909.0,75.0,79.0,1131.0,2860000
james tomkins,29.0,29.0,2560.0,1.0,1.0,7.0,915.0,51.0,71.0,1.0,1.0,803.0,2600000
mamadou sakho,27.0,27.0,2416.0,0.0,0.0,0.0,1135.0,100.0,27.0,11.0,11.0,929.0,5200000
cheikhou kouyate,31.0,21.0,1899.0,0.0,2.0,8.0,639.0,70.0,242.0,19.0,20.0,601.0,3380000


## Newcastle United

### Final Team Stats

In [53]:
newcastle_standard = standardtable(
    'https://fbref.com/en/squads/b2b47a98/2018-2019/Newcastle-United-Stats')
newcastle_shooting = shootingtable(
    'https://fbref.com/en/squads/b2b47a98/2018-2019/Newcastle-United-Stats')
newcastle_passing = passingtable(
    'https://fbref.com/en/squads/b2b47a98/2018-2019/Newcastle-United-Stats')
newcastle_possession = possessiontable(
    'https://fbref.com/en/squads/b2b47a98/2018-2019/Newcastle-United-Stats')
newcastle_salary = salarytable(
    'https://www.spotrac.com/epl/newcastle-united-f.c/payroll/2018/')

In [54]:
#merging newcastle tables into one table
newcastle_final = teamfinaltable(newcastle_standard, newcastle_shooting,
                                 newcastle_passing, newcastle_possession, newcastle_salary)

In [55]:
newcastle_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
martin dubravka,38.0,38.0,3420.0,0.0,0.0,0.0,686.0,0.0,0.0,1.0,1.0,438.0,1976000
matt ritchie,36.0,35.0,2928.0,2.0,8.0,7.0,1036.0,216.0,788.0,17.0,17.0,1014.0,2340000
ayoze perez,37.0,34.0,2917.0,12.0,2.0,26.0,609.0,68.0,562.0,63.0,67.0,834.0,2340000
jamaal lascelles,32.0,32.0,2745.0,0.0,1.0,1.0,836.0,53.0,53.0,3.0,3.0,685.0,2080000
salomon rondon,32.0,30.0,2603.0,11.0,7.0,29.0,538.0,55.0,563.0,20.0,24.0,892.0,NaN
deandre yedlin,29.0,28.0,2473.0,1.0,2.0,2.0,784.0,156.0,455.0,10.0,10.0,635.0,1560000
mohamed diame,29.0,24.0,2166.0,0.0,0.0,3.0,571.0,45.0,192.0,23.0,27.0,581.0,2340000
fabian schar,24.0,22.0,1999.0,4.0,1.0,7.0,699.0,77.0,110.0,12.0,13.0,765.0,2080000
paul dummett,26.0,21.0,1990.0,0.0,0.0,1.0,639.0,104.0,164.0,2.0,3.0,537.0,1820000


## Bournemouth

### Final Team Stats

In [56]:
bournemouth_standard = standardtable(
    'https://fbref.com/en/squads/4ba7cbea/2018-2019/Bournemouth-Stats')
bournemouth_shooting = shootingtable(
    'https://fbref.com/en/squads/4ba7cbea/2018-2019/Bournemouth-Stats')
bournemouth_passing = passingtable(
    'https://fbref.com/en/squads/4ba7cbea/2018-2019/Bournemouth-Stats')
bournemouth_possession = possessiontable(
    'https://fbref.com/en/squads/4ba7cbea/2018-2019/Bournemouth-Stats')
bournemouth_salary = salarytable(
    'https://www.spotrac.com/epl/afc-bournemouth/payroll/2018/')

In [57]:
#merging bournemouth tables into one table
bournemouth_final = teamfinaltable(bournemouth_standard, bournemouth_shooting,
                                   bournemouth_passing, bournemouth_possession, bournemouth_salary)

In [58]:
bournemouth_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
nathan ake,38.0,38.0,3411.0,4.0,0.0,8.0,1469.0,91.0,142.0,3.0,3.0,1283.0,2080000
ryan fraser,38.0,35.0,3166.0,7.0,14.0,24.0,1019.0,225.0,947.0,22.0,25.0,1274.0,1404000
joshua king,35.0,34.0,2961.0,12.0,3.0,18.0,607.0,94.0,703.0,73.0,79.0,1002.0,2340000
steve cook,31.0,31.0,2786.0,1.0,1.0,8.0,1222.0,117.0,146.0,4.0,4.0,1093.0,2340000
callum wilson,30.0,29.0,2528.0,14.0,9.0,27.0,319.0,45.0,419.0,22.0,27.0,580.0,2080000
jefferson lerma,30.0,29.0,2506.0,2.0,0.0,8.0,1320.0,117.0,313.0,28.0,28.0,1168.0,2600000
david brooks,30.0,29.0,2261.0,7.0,5.0,21.0,649.0,110.0,514.0,42.0,49.0,870.0,1040000
adam smith,25.0,25.0,2066.0,1.0,1.0,1.0,860.0,116.0,375.0,18.0,19.0,753.0,1820000
asmir begovic,24.0,24.0,2160.0,0.0,0.0,0.0,431.0,1.0,2.0,0.0,0.0,208.0,3640000


## Burnley

### Final Team Stats

In [59]:
burnley_standard = standardtable(
    'https://fbref.com/en/squads/943e8050/2018-2019/Burnley-Stats')
burnley_shooting = shootingtable(
    'https://fbref.com/en/squads/943e8050/2018-2019/Burnley-Stats')
burnley_passing = passingtable(
    'https://fbref.com/en/squads/943e8050/2018-2019/Burnley-Stats')
burnley_possession = possessiontable(
    'https://fbref.com/en/squads/943e8050/2018-2019/Burnley-Stats')
burnley_salary = salarytable(
    'https://www.spotrac.com/epl/burnley-f.c/payroll/2018/')

In [60]:
#merging burnley tables into one table
burnley_final = teamfinaltable(burnley_standard, burnley_shooting,
                               burnley_passing, burnley_possession, burnley_salary)

In [61]:
burnley_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
ben mee,38.0,38.0,3420.0,0.0,2.0,6.0,1119.0,99.0,87.0,9.0,9.0,955.0,2860000
jack cork,37.0,37.0,3312.0,1.0,2.0,2.0,1081.0,95.0,332.0,27.0,29.0,996.0,1976000
charlie taylor,38.0,35.0,3243.0,0.0,1.0,0.0,1224.0,163.0,579.0,41.0,43.0,1008.0,1300000
james tarkowski,35.0,35.0,3086.0,3.0,1.0,8.0,790.0,102.0,76.0,7.0,9.0,703.0,2600000
ashley westwood,34.0,31.0,2824.0,2.0,7.0,7.0,1111.0,200.0,458.0,11.0,11.0,1013.0,1612000
chris wood,38.0,29.0,2588.0,10.0,2.0,23.0,406.0,26.0,487.0,14.0,15.0,725.0,2600000
ashley barnes,37.0,26.0,2388.0,12.0,2.0,27.0,346.0,45.0,450.0,11.0,12.0,631.0,2080000
jeff hendrick,32.0,25.0,2063.0,3.0,0.0,6.0,511.0,59.0,327.0,14.0,18.0,557.0,1820000
johann berg gudmundsson,29.0,19.0,1752.0,3.0,6.0,11.0,407.0,87.0,455.0,18.0,19.0,497.0,1560000


## Southampton

### Final Team Stats

In [62]:
southampton_standard = standardtable(
    'https://fbref.com/en/squads/33c895d4/2018-2019/Southampton-Stats')
southampton_shooting = shootingtable(
    'https://fbref.com/en/squads/33c895d4/2018-2019/Southampton-Stats')
southampton_passing = passingtable(
    'https://fbref.com/en/squads/33c895d4/2018-2019/Southampton-Stats')
southampton_possession = possessiontable(
    'https://fbref.com/en/squads/33c895d4/2018-2019/Southampton-Stats')
southampton_salary = salarytable(
    'https://www.spotrac.com/epl/southampton-f.c/payroll/2018/')

In [63]:
#merging southampton tables into one table
southampton_final = teamfinaltable(southampton_standard, southampton_shooting,
                                   southampton_passing, southampton_possession, southampton_salary)

In [64]:
southampton_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
nathan redmond,38.0,36.0,3270.0,6.0,4.0,22.0,877.0,111.0,970.0,107.0,116.0,1231.0,3120000
pierre hojbjerg,31.0,31.0,2760.0,4.0,3.0,19.0,1495.0,165.0,532.0,30.0,31.0,1429.0,NaN
alex mccarthy,25.0,25.0,2250.0,0.0,0.0,0.0,318.0,0.0,1.0,1.0,1.0,212.0,2600000
oriol romeu,31.0,25.0,2136.0,1.0,0.0,4.0,898.0,130.0,190.0,12.0,12.0,765.0,3380000
ryan bertrand,24.0,24.0,2160.0,1.0,0.0,2.0,795.0,161.0,539.0,7.0,11.0,688.0,3640000
jan bednarek,25.0,24.0,2151.0,0.0,1.0,1.0,591.0,71.0,36.0,2.0,2.0,471.0,1040000
jannik vestergaard,23.0,23.0,2027.0,0.0,0.0,1.0,671.0,73.0,61.0,12.0,12.0,539.0,3120000
danny ings,24.0,23.0,1651.0,7.0,3.0,19.0,287.0,42.0,325.0,34.0,37.0,463.0,3900000
james ward-prowse,26.0,21.0,1945.0,7.0,0.0,14.0,622.0,88.0,482.0,6.0,7.0,596.0,2340000


## Brighton

### Final Team Stats

In [65]:
brighton_standard = standardtable(
    'https://fbref.com/en/squads/d07537b9/2018-2019/Brighton-and-Hove-Albion-Stats'
)
brighton_shooting = shootingtable(
    'https://fbref.com/en/squads/d07537b9/2018-2019/Brighton-and-Hove-Albion-Stats'
)
brighton_passing = passingtable(
    'https://fbref.com/en/squads/d07537b9/2018-2019/Brighton-and-Hove-Albion-Stats'
)
brighton_possession = possessiontable(
    'https://fbref.com/en/squads/d07537b9/2018-2019/Brighton-and-Hove-Albion-Stats'
)
brighton_salary = salarytable(
    'https://www.spotrac.com/epl/brighton-hove-albion/payroll/2018/'
)

In [66]:
#merging brighton tables into one table
brighton_final = teamfinaltable(brighton_standard, brighton_shooting,
                                brighton_passing, brighton_possession,
                                brighton_salary)

In [67]:
brighton_final

,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
index,,,,,,,,,,,,,
lewis dunk,36.0,36.0,3151.0,2.0,1.0,4.0,1240.0,50.0,83.0,1.0,1.0,1061.0,2340000
shane duffy,35.0,35.0,3087.0,5.0,1.0,14.0,990.0,33.0,112.0,5.0,5.0,918.0,2080000
mathew ryan,34.0,34.0,3060.0,0.0,0.0,0.0,551.0,0.0,0.0,0.0,0.0,395.0,1820000
davy propper,30.0,30.0,2546.0,1.0,1.0,2.0,980.0,130.0,339.0,36.0,40.0,975.0,2340000
glenn murray,38.0,30.0,2508.0,13.0,1.0,16.0,385.0,36.0,416.0,14.0,16.0,731.0,1560000
solly march,35.0,30.0,2463.0,1.0,5.0,9.0,597.0,86.0,635.0,43.0,48.0,812.0,1560000
dale stephens,30.0,29.0,2485.0,1.0,1.0,2.0,1145.0,126.0,193.0,11.0,11.0,1010.0,1560000
martin montoya,25.0,24.0,2167.0,0.0,1.0,1.0,895.0,148.0,315.0,12.0,15.0,717.0,2600000
pascal gross,25.0,24.0,1864.0,3.0,3.0,7.0,654.0,118.0,473.0,10.0,10.0,739.0,2600000


# Final Merge and Clean on 2018-2019 Stats

## Merging Teams Into One Table

In [68]:
def totalmerge():
    '''
    Merging all team_final tables (dataframe) into one table (dataframe)
    '''
    teams = [
        manutd_final,
        mancity_final,
        liverpool_final,
        chelsea_final,
        tottenham_final,
        arsenal_final,
        wolves_final,
        everton_final,
        leicester_final,
        westham_final,
        watford_final,
        crystalpalace_final,
        newcastle_final,
        bournemouth_final,
        burnley_final,
        southampton_final,
        brighton_final,
    ]
    total_1819 = pd.concat(teams)

    return total_1819

In [69]:
total_1819 = totalmerge()

## Final Data Cleaning

In [70]:
#dropping players that never played in a premier league match in the 2018-2019 season
total_1819 = total_1819[~(total_1819['MP'] == 0)]

#changing all columns to float
total_1819 = total_1819.apply(pd.to_numeric, errors='coerce')

#changing NaN to 0
total_1819 = total_1819.fillna(0)

In [71]:
#checking for players that played on two different premier league teams in the same year due to being loaned out
premleague = total_1819.reset_index()

#example of a loaned out player
premleague[premleague['index'] == 'nathaniel clyne']

,index,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
69,nathaniel clyne,4.0,1.0,111.0,0.0,0.0,0.0,68.0,10.0,34.0,0.0,0.0,58.0,0.0
335,nathaniel clyne,14.0,13.0,1143.0,0.0,1.0,0.0,454.0,76.0,206.0,6.0,6.0,359.0,3640000.0


In [72]:
#merging stats for loaned out players
premleague = premleague.groupby('index').sum().reset_index()

In [73]:
premleague[premleague['index'] == 'nathaniel clyne']

,index,MP,Starts,Min,Gls,Ast,ShotonTarget,TotPassCmp,Prog,Touches_Att3rd,DribSucc,DribPl,Rec,Annual_Salary(GBP)
311,nathaniel clyne,18.0,14.0,1254.0,0.0,1.0,0.0,522.0,86.0,240.0,6.0,6.0,417.0,3640000.0


# Exporting 2018-2019 Stats

In [74]:
premleague.to_csv('premleague_1819_stats.csv')